In [2]:
# Import libraries
import pandas as pd
from pandas import DataFrame
import numpy as np


1) UMLS_id = UMLS concept id as found on the drug label
2) concept_name =
3) detection_method = method of detection (NLP_indication/NLP_precondition/text_mention)
the authors used NLP on drug label inserts to find the side effects for a particular drug
4) concept_type = MedDRA concept type (LLT= lowest level term, PT= preferred term
in few cases, the term is neither LLT nor PT
NOTE: from medra.tsv file (SIDER website), PT covers all the different forma of a condition.
eg. the PT, abdominal distension covers the different forms namely abdominal distension, distended abdomen, swollen abdomen, swelling abdomen or swelling abd
5) UMLS_id_medra = UMLS concept id for MedDRA term
6) medra_conceptname= MedDRA concept name
IN meddra_all_indications.tsv.gz= all side effects found on the lables are given as LLT. there is atleast one PT for every LLT, but sometimes the PT is the same as the LLT

What is selected for analysis from meddra_df or meddra_all_indications.tsv?


In [3]:
# Import the datafiles- snap dataset from stanford and SIDER database
#MEDRA is a dictionary used by SIDER to extract side effects info from insert labels
meddra_df = pd.read_table('data/meddra_all_indications.tsv', sep='\t', \
                          names=["stitch_id", "UMLS_id", "detection_method", "concept_name", \
                                 "concept_type", "UMLS_id_medra", "medra_conceptname"])


In [23]:
meddra_df.head(30)

,stitch_id,UMLS_id,detection_method,concept_name,concept_type,UMLS_id_medra,medra_conceptname
0,CID100000085,C0015544,text_mention,Failure to Thrive,LLT,C0015544,Failure to thrive
1,CID100000085,C0015544,text_mention,Failure to Thrive,PT,C0015544,Failure to thrive
2,CID100000085,C0020615,text_mention,Hypoglycemia,LLT,C0020615,Hypoglycaemia
3,CID100000085,C0020615,text_mention,Hypoglycemia,PT,C0020615,Hypoglycaemia
4,CID100000085,C0022661,NLP_indication,"Kidney Failure, Chronic",LLT,C0022661,Renal failure chronic
5,CID100000085,C0022661,NLP_indication,"Kidney Failure, Chronic",PT,C0022661,Renal failure chronic
6,CID100000085,C0025521,NLP_indication,Inborn Errors of Metabolism,LLT,C0025521,Inborn error of metabolism
7,CID100000085,C0025521,NLP_indication,Inborn Errors of Metabolism,PT,C0025521,Inborn error of metabolism
8,CID100000085,C0026827,text_mention,Muscle hypotonia,LLT,C0026827,Hypotonia
9,CID100000085,C0026827,text_mention,Muscle hypotonia,PT,C0026827,Hypotonia


In [5]:
meddra_df.shape

(30835, 7)

In [6]:
meddra_label_df =pd.read_table('data/meddra_all_label_indications.tsv', sep='\t', \
                               names=["source_label", "stitch_id1", "stitch_id2","UMLS_id", \
                                      "detection_method", "concept_name", \
                                 "concept_type", "UMLS_id_medra", "medra_conceptname"])
#What is the difference between stitch_id1 and "stitch_id2"?
# there is only a difference of 1 digit- stitch_id1 is CID1... and stitch_id2 is CID0...
#CIDs / CID0... - this is a stereo-specific compound, and the suffix is the 
#PubChem compound id.
#CIDm / CID1... - this is a "flat" compound, i.e. with merged stereo-isomers
#The suffix (without the leading "1") is the PubChem compound id.

In [8]:
meddra_label_df.head(20)

,source_label,stitch_id1,stitch_id2,UMLS_id,detection_method,concept_name,concept_type,UMLS_id_medra,medra_conceptname
0,EMA/WC500020092.html,CID100216416,CID000216416,C0016658,NLP_indication,Fracture,LLT,C0016658,Fracture
1,EMA/WC500020092.html,CID100216416,CID000216416,C0016658,NLP_indication,Fracture,PT,C0016655,Multiple fractures
2,EMA/WC500020092.html,CID100216416,CID000216416,C0016658,NLP_indication,Fracture,PT,C0016658,Fracture
3,EMA/WC500020092.html,CID100216416,CID000216416,C0019557,NLP_indication,Hip Fractures,LLT,C0019557,Hip fracture
4,EMA/WC500020092.html,CID100216416,CID000216416,C0019557,NLP_indication,Hip Fractures,PT,C0019557,Hip fracture
5,EMA/WC500020092.html,CID100216416,CID000216416,C0029456,NLP_indication,Osteoporosis,LLT,C0029456,Osteoporosis
6,EMA/WC500020092.html,CID100216416,CID000216416,C0029456,NLP_indication,Osteoporosis,PT,C0029456,Osteoporosis
7,EMA/WC500020092.html,CID100216416,CID000216416,C0236075,text_mention,Menopausal symptoms,LLT,C0236075,Menopausal symptoms
8,EMA/WC500020092.html,CID100216416,CID000216416,C0236075,text_mention,Menopausal symptoms,PT,C0236075,Menopausal symptoms
9,EMA/WC500020170.html,CID100060795,CID000060795,C0005586,NLP_precondition,Bipolar Disorder,LLT,C0005586,Bipolar disorder


In [10]:
meddra_label_df.shape

(447150, 9)

In [11]:
medra_label_big=pd.read_table('data/meddra_all_label_se.tsv', sep='\t', \
                               names=["source_label", "stitch_id1", "stitch_id2","UMLS_id", \
                                 "concept_type", "UMLS_id2", "concept_name"])
#medra_label_big is file in which concept name is repeated since there are variants for 
#side effects or concept_name- eg. abdominal pain can be gastrointestinal pain. 

In [12]:
medra_label_big.shape

(4753141, 7)

In [13]:
medra_label_big.head(10)

,source_label,stitch_id1,stitch_id2,UMLS_id,concept_type,UMLS_id2,concept_name
0,EMA/WC500020092.html,CID100216416,CID000216416,C0000737,LLT,C0000737,Abdominal pain
1,EMA/WC500020092.html,CID100216416,CID000216416,C0000737,PT,C0000737,Abdominal pain
2,EMA/WC500020092.html,CID100216416,CID000216416,C0000737,PT,C0687713,Gastrointestinal pain
3,EMA/WC500020092.html,CID100216416,CID000216416,C0002170,LLT,C0002170,Alopecia
4,EMA/WC500020092.html,CID100216416,CID000216416,C0002170,PT,C0002170,Alopecia
5,EMA/WC500020092.html,CID100216416,CID000216416,C0002395,LLT,C0002395,Dementia Alzheimer's type
6,EMA/WC500020092.html,CID100216416,CID000216416,C0002395,PT,C0002395,Dementia Alzheimer's type
7,EMA/WC500020092.html,CID100216416,CID000216416,C0002622,LLT,C0002622,Amnesia
8,EMA/WC500020092.html,CID100216416,CID000216416,C0002622,PT,C0002622,Amnesia
9,EMA/WC500020092.html,CID100216416,CID000216416,C0002871,LLT,C0002871,Anaemia


1) cells not needed : detection_method, UMLS_id
2)UMLS id - concept unique identifier for the concept_name. for eg.if you search for C0016658	UMLS_id, it will show concept_name associated with it is fracture.  (https://ncim.nci.nih.gov/ncimbrowser/ConceptReport.jsp?dictionary=NCI%20Metathesaurus&code=C00166580)

In [14]:
bio_decagon=pd.read_csv('data/bio-decagon-combo.csv')

In [15]:
bio_decagon.head()

,STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache


In [16]:
bio_decagon.shape

(4649441, 4)

In [17]:
drug_names=pd.read_table('data/drug_names.tsv', sep='\t', header=None, names=["stitch_code", "drug_name"])

In [18]:
drug_names.head()

,stitch_code,drug_name
0,CID100000085,carnitine
1,CID100000119,gamma-aminobutyric
2,CID100000137,5-aminolevulinic
3,CID100000143,leucovorin
4,CID100000146,5-methyltetrahydrofolate


In [19]:
drug_names.shape

(1430, 2)

In [20]:
drug_meddra=pd.merge(drug_names, meddra_df, left_on="stitch_code", right_on="stitch_id")

In [21]:
drug_meddra.head()

,stitch_code,drug_name,stitch_id,UMLS_id,detection_method,concept_name,concept_type,UMLS_id_medra,medra_conceptname
0,CID100000085,carnitine,CID100000085,C0015544,text_mention,Failure to Thrive,LLT,C0015544,Failure to thrive
1,CID100000085,carnitine,CID100000085,C0015544,text_mention,Failure to Thrive,PT,C0015544,Failure to thrive
2,CID100000085,carnitine,CID100000085,C0020615,text_mention,Hypoglycemia,LLT,C0020615,Hypoglycaemia
3,CID100000085,carnitine,CID100000085,C0020615,text_mention,Hypoglycemia,PT,C0020615,Hypoglycaemia
4,CID100000085,carnitine,CID100000085,C0022661,NLP_indication,"Kidney Failure, Chronic",LLT,C0022661,Renal failure chronic
